Perhaps one of the most important aspects of building a continuous stream of new information associated with species in the FWS work plan or other circumstances is the ability to identify and access new published literature. Many of the structured databases assembled and organized into this collection are based wholly or partly in literature references and information extracted from literature. We are currently working with the team developing the xDD Digital Library on a number of tools and techniques for a) identifying literature potentially applicable to species-based research and b) using natural language processing tools to pull specific data from those sources for use. This is an ongoing effort that will result in improved production capabilities over time.

In the near term, we take advantage of some basic and enhanced search functionality to identify potential articles of interested in the xDD library of millions of documents that are increasing daily. The xdd module in the bispy package contains some search and packaging functionality that interfaces with the xDD REST API.

# Data Management Considerations
The xDD Library aspect of this overall workflow is likely the most dynamic area of the system we are building as new literature is coming out all the time and the xDD Library is continuing to expand rapidly in its work to index and process as many articles as possible. Through our partnership with University of Wisconsin-Madison and the xDD team, we also have the ability to collaborate on incorporating new sources of papers and meaningful gray literature as we dig further into sources of references that may not have yet found a route into the xDD processing engine.

In [1]:
import requests
import bispy
from IPython.display import display
from joblib import Parallel, delayed
import jsonschema

xdd = bispy.xdd.Xdd()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
name_list = helperfunctions.workplan_species()

In [3]:
%%time
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
xdd_results = Parallel(n_jobs=8)(delayed(xdd.snippets)(name) for name, name_source in name_list)


CPU times: user 1.03 s, sys: 131 ms, total: 1.16 s
Wall time: 3min 26s


In [4]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("../cache/xdd.json", xdd_results))

{'Doc Cache File': '../cache/xdd.json',
 'Number of Documents in Cache': 386,
 'Document Number 322': {'processing_metadata': {'status': 'failure',
   'date_processed': '2019-09-16T18:36:00.022261',
   'api': 'https://geodeepdive.org/api/snippets?full_results&clean&term=Rhaphiomidas trochilus',
   'status_message': 'No data returned.'},
  'parameters': {'Search Term': 'Rhaphiomidas trochilus'}}}

In [5]:
xdd_schema = helperfunctions.load_schema('xdd')
display(xdd_schema)

jsonschema.validate(xdd_results,xdd_schema)

{'definitions': {'items': {'$id': '#/items',
   'type': ['object', 'array'],
   'title': 'Generic container for items in a dataset',
   'description': 'A JSON array or object property containing one or more items in a dataset or data structure within a dataset.'}},
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'http://data.usgs.gov/property_registry/',
 'type': 'array',
 'title': 'xDD Documents',
 'description': 'A dataset that contains results of a directed search against the xDD (formerly GeoDeepDive) digital library (geodeepdive.org) for a given search term.',
 'items': {'$ref': '#/definitions/items',
  'required': ['processing_metadata'],
  'properties': {'processing_metadata': {'$ref': 'common_properties.json#/definitions/processing_metadata'},
   'parameters': {'$ref': 'common_properties.json#/definitions/parameters',
    'properties': {'Search Term': {'$id': '#/items/properties/parameters/properties/Search Term',
      'type': 'string',
      'title': 'Search Te